### Exercise 4 of Data Mining class

#### João Pedro Moreto Lourenção

In [212]:
import pandas as pd

df = pd.read_csv('bebe-baixo-peso.csv')
df.head()

,Id,platelet_count,Race,Birth_weight,Gestational_age,Mother_treated_with_magnesiun_sulfate,Mother_treated_with_beta-methasone,Mother_treated_with_beta-adrenergic_drug,Delivery_procedure,Apgar_score,ventilation,Pneumothorax,Patent_ductus_arteriosus,Peri-ventricular_haemohrrage,Intra-ventricular_haemorrhage,Gender,Dead
0,1,100.0,white,1250.0,35.0,NaN,no,no,Caesarean,8.0,no,no,no,NaN,NaN,female,no
1,2,244.0,white,1370.0,32.0,NaN,yes,no,Caesarean,7.0,no,no,no,NaN,NaN,female,no
2,3,114.0,black,620.0,23.0,NaN,no,yes,natural,1.0,yes,no,no,NaN,NaN,female,yes
3,4,182.0,black,1480.0,32.0,NaN,yes,no,natural,8.0,no,no,no,NaN,NaN,male,no
4,5,54.0,black,925.0,28.0,NaN,no,no,Caesarean,5.0,yes,yes,no,definite,definite,female,yes


Filtrando o dataset para analisar as colunas de interesse.

In [213]:
from mlxtend.frequent_patterns import apriori, association_rules

cols_of_interest = [
    'Mother_treated_with_beta-methasone', 
    'Dead',
    'Race'
]

df_filtered = df[cols_of_interest]

df_dummies = pd.get_dummies(df_filtered)

df_dummies.head()


,Mother_treated_with_beta-methasone_no,Mother_treated_with_beta-methasone_yes,Dead_no,Dead_yes,Race_black,Race_native-American,Race_oriental,Race_white
0,True,False,True,False,False,False,False,True
1,False,True,True,False,False,False,False,True
2,True,False,False,True,True,False,False,False
3,False,True,True,False,True,False,False,False
4,True,False,False,True,True,False,False,False


In [214]:
frequent_itemsets = apriori(df_dummies, min_support=0.2, use_colnames=True)

frequent_itemsets

,support,itemsets
0,0.473920,(Mother_treated_with_beta-methasone_no)
1,0.368107,(Mother_treated_with_beta-methasone_yes)
2,0.785395,(Dead_no)
3,0.214605,(Dead_yes)
4,0.549925,(Race_black)
5,0.383010,(Race_white)
6,0.363636,"(Mother_treated_with_beta-methasone_no, Dead_no)"
7,0.286140,"(Mother_treated_with_beta-methasone_no, Race_b..."
8,0.324888,"(Mother_treated_with_beta-methasone_yes, Dead_no)"
9,0.424739,"(Race_black, Dead_no)"


In [215]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

# b
beta_methasone_rules = rules[
    rules['antecedents'].apply(lambda x: 'Mother_treated_with_beta-methasone_yes' in x)
]

# c
race_death_rules = rules[
    rules['antecedents'].apply(lambda x: 'Race_black' in x) &
    rules['consequents'].apply(lambda x: 'Dead_yes' in x or 'Dead_no' in x)
]

print(f'O número de regras encontrado foi: {len(rules)}')

O número de regras encontrado foi:9


In [216]:
beta_methasone_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
3,(Mother_treated_with_beta-methasone_yes),(Dead_no),0.368107,0.785395,0.324888,0.882591,1.123755,0.035779,1.827843,0.174279


In [217]:
race_death_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
4,(Race_black),(Dead_no),0.549925,0.785395,0.424739,0.772358,0.983400,-0.007169,0.942729,-0.036149
7,"(Mother_treated_with_beta-methasone_no, Race_b...",(Dead_no),0.286140,0.785395,0.205663,0.718750,0.915145,-0.019070,0.763040,-0.114958


#### Análise
A partir das 9 regras obtidas, 3 delas foram selecionadas para a análise. A primeira delas mostra, com uma confiança de 88,25%, que o uso de beta-methasone pela mãe, resulta na não morte do bebê. 

O segundo conjunto de dados mostra, com 77,23% de confiança, que bebês afro-descendentes com baixo peso sobrevivem. Além disso, mostra também, com 71,87% de confiança, que bebês afro-descendentes cujas mães não tomaram beta-methasone, sobreviveram. É possível afirmar que o uso da beta-methasone melhora o índice de sobrevivência de bebês baixo peso, porém aparentemente não tem o mesmo efeito com os bebês afro-descendentes. 

Dados encontrados na internet indicam que existe evidência robusta que os corticosteroides:

- reduzem o risco de mortes perinatais (bebês que nascem mortos e bebês que morrem nos primeiros 28 dias de vida);

- reduzem o risco de mortes neonatais (bebês que morrem nos primeiros 28 dias de vida);

- reduzem o risco de problemas respiratórios graves nas primeiras horas de vida;

- têm pouco ou nenhum efeito no peso de nascimento dos bebês.

Os corticosteroides provavelmente reduzem o risco de:

- sangramento dentro do cérebro;

- atraso no desenvolvimento na infância.

- os bebês nos estudos podem não ter sido representativos de todos os bebês prematuros; ou

- é possível que os estudos tenham sido conduzidos de forma a introduzir erros nos seus resultados.

Quanto à saúde materna, as evidências indicam que os corticosteroides provavelmente não têm efeitos sobre o risco de:

- morte materna após o parto;

- desenvolver corioamnionite (inflamação ou infecção dos tecidos que envolvem o bebê no útero);

- desenvolver endometrite (inflamação do revestimento interno do útero).